In [1]:
# Load libraries
import folium
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

## Support Functions

In [2]:
def plot_value_unplug_trips(value):
    """
    Support function to define color scale for Madrid Bike station map.
    """
    if value < 300:
        return 1
    elif value >= 300 and value < 800:
        return 2
    elif value >= 800 and value < 1500:
        return 3
    elif value >= 1500 and value < 3500:
        return 4
    elif value >= 3500 and value < 6500:
        return 5
    else:
        return 6

## Load Dataset

In [3]:
# Load files
full_dataset = "plot_map_bicimad"

bike_full_df = pd.read_csv(full_dataset, header = 0, sep = ';')

In [4]:
# Train, test split stratify based on ageRange
bike_full_df, bike_full_df_test = train_test_split(bike_full_df, stratify=bike_full_df["ageRange"],test_size=0.20, random_state=123)

In [5]:
# Removing Outliers (!Note to be deleted once fixed pipeline)
bike_full_df = bike_full_df[bike_full_df['travel_time_min']<=30]

In [ ]:
print(bike_full_df.shape)
bike_full_df.head()

## Plot BiciMAD Stations

In [7]:
# Load Madrid map
madrid_bicimad_map = folium.Map(location=[40.4168, -3.7038], zoom_start=12)

# Extract unique BiciMAD stations latitude and longitude values from the bike_full_df for year 2019
unique_unplug_station = bike_full_df[['idunplug_latitude', 'idunplug_longitude']].drop_duplicates()


# Add markers to the map for each unique station location
for idx, row in unique_unplug_station.iterrows():
    lat, lon = row['idunplug_latitude'], row['idunplug_longitude']
    folium.CircleMarker(
        location=[lat, lon],
        radius=2,  # Adjust the size of the markers as needed
        color='black',
        fill=True,
        fill_color='black'
    ).add_to(madrid_bicimad_map)

# Create plot custom legend
legend_html = '''
     <div style="position: fixed; 
                 bottom: 50px; left: 50px; width: 200px; height: 50px; 
                 border:2px solid grey; z-index:9999; font-size:14px;
                 background-color: white;">
     &nbsp; <b>Legend:</b> <br>
     &nbsp; <i class="fas fa-circle" style="color:black"></i> BiciMAD Stations (2019) <br>
     </div>
     '''


# Add legend to the map
madrid_bicimad_map.get_root().html.add_child(folium.Element(legend_html))

# Save the BiciMAD station map as an HTML file
madrid_bicimad_map.save('madrid_bicimad_stations.html')


## Plot BiciMAD Generated Trips from Unplug stations by ageRange

### Plot Age Group 3: 

In [7]:
# Filter dataframe based on relevant Age Group: 3
bike_age_3_df =  bike_full_df[bike_full_df["ageRange"].isin([3])]

In [ ]:
# Grouping by idunplug_latitude and idunplug_longitude and counting occurrences
tot_trips_unplug_3 = bike_age_3_df.groupby(['idunplug_latitude', 'idunplug_longitude']).size().reset_index(name='tot_trips')

# Add support plot_val col to be used for defining color in viz.
tot_trips_unplug_3['plot_val'] = tot_trips_unplug_3['tot_trips'].apply(plot_value_unplug_trips)

# Displaying tot. trips per station for selected AgeGroup
display(tot_trips_unplug_3)

In [ ]:
# Tot. # trips by age group for year 2019
tot_trips_unplug_3['tot_trips'].sum()

In [ ]:
# Tot. trips distribution per station by age group 
tot_trips_unplug_3.describe()

In [11]:
# Load Madrid map
madrid_map_age_3 = folium.Map(location=[40.4168, -3.7038], zoom_start=12)

# Extract latitude, longitude, and plot_val values from the tot_trips_unplug dataframe
latitudes = tot_trips_unplug_3['idunplug_latitude'].values
longitudes = tot_trips_unplug_3['idunplug_longitude'].values
values = tot_trips_unplug_3['plot_val'].values

# Define color scale based on the rules provided
color_scale = folium.StepColormap(
    colors=['gray', 'blue', 'green', 'yellow', 'red', 'purple','black'],
    index=[1, 2, 3, 4, 5, 6, 7],
    vmin=min(values),
    vmax=max(values),
    caption='Color Scale'
)

# Add markers to the map
for lat, lon, value in zip(latitudes, longitudes, values):
    folium.CircleMarker(
        location=[lat, lon],
        radius=5,  # Adjust the size of the markers as needed
        color=color_scale(value),
        fill=True,
        fill_color=color_scale(value),
        fill_opacity=0.7,
        popup=f'Value: {value}'
    ).add_to(madrid_map_age_3)

# Create a custom legend
legend_html = '''
     <div style="position: fixed; 
                 bottom: 50px; left: 50px; width: 340px; height: 160px; 
                 border:2px solid grey; z-index:9999; font-size:14px;
                 background-color: white;">
        <div style="text-align: center;">
            &nbsp; <b>Age Cohort 3 (2019)</b>
        </div>
     &nbsp; <u>Total Trip generated per unplug station:</u> <br>
     &nbsp; <i class="fas fa-circle" style="color:gray"></i> Total Trips < 300 <br>
     &nbsp; <i class="fas fa-circle" style="color:blue"></i> 300 <= Total Trips < 800 <br>
     &nbsp; <i class="fas fa-circle" style="color:green"></i> 800 <= Total Trips < 1500 <br>
     &nbsp; <i class="fas fa-circle" style="color:yellow"></i> 1500 <= Total Trips < 3500 <br>
     &nbsp; <i class="fas fa-circle" style="color:red"></i> 3500 <= Total Trips < 6500 <br>
     &nbsp; <i class="fas fa-circle" style="color:purple"></i> Total Trips >= 6500 <br>
     </div>
     '''

# Add legend to the map
madrid_map_age_3.get_root().html.add_child(folium.Element(legend_html))

# Save the map as an HTML file
madrid_map_age_3.save('madrid_trip_generated_age_cohort_3.html')


### Plot Age Group 4:

In [12]:
# Filter dataframe based on relevant Age Group: 4
bike_age_4_df =  bike_full_df[bike_full_df["ageRange"].isin([4])]

In [ ]:
# Grouping by idunplug_latitude and idunplug_longitude and counting occurrences
tot_trips_unplug_4 = bike_age_4_df.groupby(['idunplug_latitude', 'idunplug_longitude']).size().reset_index(name='tot_trips')

# Add support plot_val col to be used for defining color in viz.
tot_trips_unplug_4['plot_val'] = tot_trips_unplug_4['tot_trips'].apply(plot_value_unplug_trips)

# Displaying tot. trips per station for selected AgeGroup
display(tot_trips_unplug_4)

In [ ]:
# Tot. # trips by age group for year 2019
tot_trips_unplug_4['tot_trips'].sum()

In [ ]:
# Tot. trips distribution per station by age group 
tot_trips_unplug_4.describe()

In [16]:
# Load Madrid map
madrid_map_age_4 = folium.Map(location=[40.4168, -3.7038], zoom_start=12)

# Extract latitude, longitude, and plot_val values from the tot_trips_unplug dataframe
latitudes = tot_trips_unplug_4['idunplug_latitude'].values
longitudes = tot_trips_unplug_4['idunplug_longitude'].values
values = tot_trips_unplug_4['plot_val'].values

# Define color scale based on the rules provided
color_scale = folium.StepColormap(
    colors=['gray', 'blue', 'green', 'yellow', 'red', 'purple','black'],
    index=[1, 2, 3, 4, 5, 6, 7],
    vmin=min(values),
    vmax=max(values),
    caption='Color Scale'
)

# Add markers to the map
for lat, lon, value in zip(latitudes, longitudes, values):
    folium.CircleMarker(
        location=[lat, lon],
        radius=5,  
        color=color_scale(value),
        fill=True,
        fill_color=color_scale(value),
        fill_opacity=0.7,
        popup=f'Value: {value}'
    ).add_to(madrid_map_age_4)

# Create a custom legend
legend_html = '''
     <div style="position: fixed; 
                 bottom: 50px; left: 50px; width: 340px; height: 160px; 
                 border:2px solid grey; z-index:9999; font-size:14px;
                 background-color: white;">
        <div style="text-align: center;">
            &nbsp; <b>Age Cohort 4 (2019)</b>
        </div>
     &nbsp; <u>Total Trip generated per unplug station:</u> <br>
     &nbsp; <i class="fas fa-circle" style="color:gray"></i> Total Trips < 300 <br>
     &nbsp; <i class="fas fa-circle" style="color:blue"></i> 300 <= Total Trips < 800 <br>
     &nbsp; <i class="fas fa-circle" style="color:green"></i> 800 <= Total Trips < 1500 <br>
     &nbsp; <i class="fas fa-circle" style="color:yellow"></i> 1500 <= Total Trips < 3500 <br>
     &nbsp; <i class="fas fa-circle" style="color:red"></i> 3500 <= Total Trips < 6500 <br>
     &nbsp; <i class="fas fa-circle" style="color:purple"></i> Total Trips >= 6500 <br>
     </div>
     '''

# Add legend to the map
madrid_map_age_4.get_root().html.add_child(folium.Element(legend_html))

# Save the map as an HTML file
madrid_map_age_4.save('madrid_trip_generated_age_cohort_4.html')


### Plot Age Group 5:

In [17]:
# Filter dataframe based on relevant Age Group: 5
bike_age_5_df =  bike_full_df[bike_full_df["ageRange"].isin([5])]

In [ ]:
# Grouping by idunplug_latitude and idunplug_longitude and counting occurrences
tot_trips_unplug_5 = bike_age_5_df.groupby(['idunplug_latitude', 'idunplug_longitude']).size().reset_index(name='tot_trips')

# Add support plot_val col to be used for defining color in viz.
tot_trips_unplug_5['plot_val'] = tot_trips_unplug_5['tot_trips'].apply(plot_value_unplug_trips)

# Displaying tot. trips per station for selected AgeGroup
display(tot_trips_unplug_5)

In [ ]:
# Tot. # trips by age group for year 2019
tot_trips_unplug_5['tot_trips'].sum()

In [ ]:
# Tot. trips distribution per station by age group 
tot_trips_unplug_5.describe()

In [21]:
# Load Madrid map
madrid_map_age_5 = folium.Map(location=[40.4168, -3.7038], zoom_start=12)

# Extract latitude, longitude, and plot_val values from the tot_trips_unplug dataframe
latitudes = tot_trips_unplug_5['idunplug_latitude'].values
longitudes = tot_trips_unplug_5['idunplug_longitude'].values
values = tot_trips_unplug_5['plot_val'].values

# Define color scale based on the rules provided
color_scale = folium.StepColormap(
    colors=['gray', 'blue', 'green', 'yellow', 'red', 'purple','black'],
    index=[1, 2, 3, 4, 5, 6, 7],
    vmin=min(values),
    vmax=max(values),
    caption='Color Scale'
)

# Add markers to the map
for lat, lon, value in zip(latitudes, longitudes, values):
    folium.CircleMarker(
        location=[lat, lon],
        radius=5,  
        color=color_scale(value),
        fill=True,
        fill_color=color_scale(value),
        fill_opacity=0.7,
        popup=f'Value: {value}'
    ).add_to(madrid_map_age_5)

# Create a custom legend
legend_html = '''
     <div style="position: fixed; 
                 bottom: 50px; left: 50px; width: 340px; height: 160px; 
                 border:2px solid grey; z-index:9999; font-size:14px;
                 background-color: white;">
        <div style="text-align: center;">
            &nbsp; <b>Age Cohort 5 (2019)</b>
        </div>
     &nbsp; <u>Total Trip generated per unplug station:</u> <br>
     &nbsp; <i class="fas fa-circle" style="color:gray"></i> Total Trips < 300 <br>
     &nbsp; <i class="fas fa-circle" style="color:blue"></i> 300 <= Total Trips < 800 <br>
     &nbsp; <i class="fas fa-circle" style="color:green"></i> 800 <= Total Trips < 1500 <br>
     &nbsp; <i class="fas fa-circle" style="color:yellow"></i> 1500 <= Total Trips < 3500 <br>
     &nbsp; <i class="fas fa-circle" style="color:red"></i> 3500 <= Total Trips < 6500 <br>
     &nbsp; <i class="fas fa-circle" style="color:purple"></i> Total Trips >= 6500 <br>
     </div>
     '''

# Add legend to the map
madrid_map_age_5.get_root().html.add_child(folium.Element(legend_html))

# Save the map as an HTML file
madrid_map_age_5.save('madrid_trip_generated_age_cohort_5.html')
